In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [7]:
image_shape = 28

In [27]:
# Load the train dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    './Train_osszes',   # Path to your dataset directory
    labels='inferred',         # Automatically label based on subdirectory names
    label_mode='int',          # 'int' for integer labels, 'categorical' for one-hot labels
    color_mode="grayscale",    # only one channel
    batch_size=32,             # Number of images per batch
    image_size=(128, 128),     # Resize all images to this size
    shuffle=True,             # Shuffle the dataset
    seed=42                    # Optional seed for reproducibility
)

Found 53172 files belonging to 62 classes.


In [28]:
# Load the dataset
test_dataset = tf.keras.utils.image_dataset_from_directory(
    './TestData',   # Path to your dataset directory
    labels=None,         # Automatically label based on subdirectory names
    color_mode="grayscale",    # only one channel
    batch_size=32,             # Number of images per batch
    image_size=(128, 128),     # Resize all images to this size
    shuffle=False,             # Shuffle the dataset
    seed=42                    # Optional seed for reproducibility
)

Found 7100 files.


In [29]:
train_dataset = train_dataset.map(lambda x, y: (tf.image.resize(x, [image_shape, image_shape]), y))
test_dataset = test_dataset.map(lambda x: (tf.image.resize(x, [image_shape, image_shape])))

In [31]:
from sklearn.model_selection import KFold
# Convert dataset to numpy arrays
def dataset_to_numpy(dataset):
    images = []
    labels = []
    for image_batch, label_batch in dataset:
        images.append(image_batch.numpy())
        labels.append(label_batch.numpy())
    
    images = np.concatenate(images, axis=0)
    labels = np.concatenate(labels, axis=0)
    return images, labels

In [32]:
train_images, labels = dataset_to_numpy(train_dataset)

In [36]:
def test_dataset_to_numpy(dataset):
    images = []
    for image_batch in dataset:
        images.append(image_batch.numpy())
    
    images = np.concatenate(images, axis=0)
    return images

In [37]:
test_images = test_dataset_to_numpy(test_dataset)

In [43]:
from tensorflow.keras import layers, models
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout
from keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Define model creation function
def create_model(print_summary=False): 
    model = Sequential([
        Input(shape=(image_shape, image_shape, 1)),
        Conv2D(32, 3, activation='relu'),
        MaxPool2D((2, 2)),
        BatchNormalization(),
        Dropout(0.25),
        Conv2D(64, 3, activation='relu'),
        MaxPool2D((2, 2)),
        BatchNormalization(),
        Flatten(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dense(128, activation='relu'),
        Dropout(0.25),
        BatchNormalization(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dense(62, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    if(print_summary): model.summary()
    return model

In [49]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau,ModelCheckpoint
model = create_model()
RLP = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001)
model.fit(train_images, labels, epochs=35, batch_size=64, callbacks=[RLP])


Epoch 1/35
831/831 ━━━━━━━━━━━━━━━━━━━━ 36s 32ms/step - accuracy: 0.5650 - loss: 1.7165 - learning_rate: 0.0010
Epoch 2/35
831/831 ━━━━━━━━━━━━━━━━━━━━ 27s 33ms/step - accuracy: 0.8288 - loss: 0.5066 - learning_rate: 0.0010
Epoch 3/35
831/831 ━━━━━━━━━━━━━━━━━━━━ 27s 33ms/step - accuracy: 0.8510 - loss: 0.4086 - learning_rate: 0.0010
Epoch 4/35
831/831 ━━━━━━━━━━━━━━━━━━━━ 27s 33ms/step - accuracy: 0.8662 - loss: 0.3506 - learning_rate: 0.0010
Epoch 5/35
831/831 ━━━━━━━━━━━━━━━━━━━━ 27s 33ms/step - accuracy: 0.8761 - loss: 0.3197 - learning_rate: 0.0010
Epoch 6/35
831/831 ━━━━━━━━━━━━━━━━━━━━ 27s 32ms/step - accuracy: 0.8824 - loss: 0.2981 - learning_rate: 0.0010
Epoch 7/35
831/831 ━━━━━━━━━━━━━━━━━━━━ 27s 33ms/step - accuracy: 0.8930 - loss: 0.2725 - learning_rate: 0.0010
Epoch 8/35
831/831 ━━━━━━━━━━━━━━━━━━━━ 26s 31ms/step - accuracy: 0.8956 - loss: 0.2591 - learning_rate: 0.0010
Epoch 9/35
831/831 ━━━━━━━━━━━━━━━━━━━━ 26s 32ms/step - accuracy: 0.9036 - loss: 0.2485 - learning_rate:

In [50]:
pred = model.predict(test_images)

222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step


In [51]:
predicted_values = np.argmax(pred, axis=1) + 1

45

In [52]:
import os

image_directory = './TestData'
file_names = [f for f in os.listdir(image_directory) if f.endswith('.png')]

In [103]:
import csv

# CSV fájl létrehozása és írása
with open('OCRpredictions.csv', mode='w', newline='') as file:
    writer = csv.writer(file, delimiter=';')  # Használj ';' elválasztót
    writer.writerow(['class', 'TestImage'])  # Fejléc
    for pred, name in zip(predicted_values, file_names):
        writer.writerow([pred, name])  # Először osztály, utána fájlnév


In [105]:
import pandas as pd
pd2=pd.read_csv('./OCRpredictions.csv', delimiter=';')

In [107]:
if len(pd2)!=7100:
    print('Error! Number of predictions should be 7100!')

In [109]:
if pd2['class'][0]>62:
    print('Error!')

In [111]:
if pd2['class'].max()>62 or pd2['class'].min()<1:
    print('Error! All class labels should be between 1 and 62! First class label is not zero!')

In [113]:
if pd2['TestImage'][0]!='Test0001.png':
    print('Error! First image should be the Test0001.png.')
if pd2['TestImage'][7100-1]!='Test7100.png':
    print('Error! Last image should be the Test7100.png.')